In [ ]:
import os
import numpy as np
from sklearn.cluster import KMeans
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image as keras_image

Load and preprocess images:

In [ ]:
def load_image(image_path, target_size=(224, 224)):
    img = keras_image.load_img(image_path, target_size=target_size)
    img = keras_image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

image_dir = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\images'
image_files = os.listdir(image_dir)

images = [load_image(os.path.join(image_dir, f)) for f in image_files]
images = np.vstack(images)

Load pre-trained ResNet50 model and extract features:

In [ ]:
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

features = model.predict(images)

implement the elbow method to find optimal clusters number

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Calculate WCSS for different numbers of clusters
wcss = []
min_range = 1
max_range = 88
for i in range(min_range, max_range):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(features)
    wcss.append(kmeans.inertia_)

# Plot the WCSS values
plt.plot(range(min_range, max_range), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

Apply K-means clustering:

In [ ]:
kmeans = KMeans(n_clusters=15, random_state=42)
clusters = kmeans.fit_predict(features)

Group images by cluster:

In [ ]:
clustered_images = {}
for i, cluster in enumerate(clusters):
    if cluster not in clustered_images:
        clustered_images[cluster] = []
    clustered_images[cluster].append(image_files[i])

In [ ]:
print(clusters)
print(clusters.shape)

In [ ]:
import cv2 as OpenCV
import matplotlib.pyplot as plt

def display_rgb_image(path, title):
    rgb_image = OpenCV.cvtColor(OpenCV.imread(path), OpenCV.COLOR_BGR2RGB)
    plt.imshow(rgb_image)
    if title is not None:
        plt.title(title)
    plt.axis('off')
    plt.show()

def display_similar_images(key, values):
    print(f"cluster {key}")
    print("-----------------------------------------------------")
    for value in values:
            print(value)
            display_rgb_image(os.path.join(image_dir, value), value)

In [ ]:
print(clustered_images.keys())

In [ ]:
print(sum(len(clustered_images[i]) for i in clustered_images.keys()))

In [ ]:
for key, values in clustered_images.items():
    display_similar_images(key, values)